In [1]:
import torch
import torchvision
import torch.nn as nn
import time
import json
import datetime
from DeepLearning.Project1.data import *

In [2]:
# File based on https://github.com/huyvnphan/PyTorch_CIFAR10/
class VGG(nn.Module):
    def __init__(self, features, num_classes=10, avgpool_size=(1,1)):
        super(VGG, self).__init__()
        self.features = features

        self.avgpool = nn.AdaptiveAvgPool2d(avgpool_size)

        self.classifier = nn.Sequential(
            nn.Linear(512 * avgpool_size[0] * avgpool_size[1], 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

 # Create convolution part of VGG11_bn archhitecture
def make_vgg11_bn_layers(cfg = None):
    if cfg == None:
        cfg = [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"]
    layers = []
    in_channels = 3
    for v in cfg:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

 # Create VGG11_bn model
def vgg11_bn(device="cpu", num_classes=10):
    model = VGG(make_vgg11_bn_layers(), num_classes=num_classes)
    return model

vgg_cfg = {
    '8':  [64, 'M', 128, 'M', 256, 'M', 512, 'M', 512, 'M'],
    '11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    '13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    '16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    '19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 
          512, 512, 512, 512, 'M'],
}



def eval_accuracy(model, dataloader, training_device='cpu'):
    with torch.no_grad():
        model.to(training_device)
        correct = 0
        all_so_far = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(training_device), labels.to(training_device)
            pred = torch.argmax(model(inputs), dim=1)

            all_so_far += labels.size().numel()
            correct += torch.sum(pred.eq(labels)).item()
    return correct/all_so_far

In [3]:
def backup_to_ram(model):
    from copy import deepcopy
    return deepcopy(model).cpu()

class EarlyStopper:
    def __init__(self, patience = 3, backup_method=backup_to_ram):
        self.patience = patience
        self.current = 0
        
        self.backup_method = backup_method
        
        self.best_backup = None
        self.best_accuracy = 0.

    def should_continue(self, accuracy, model = None):
        if self.best_accuracy < accuracy:
            self.current = 0
            self.best_accuracy = accuracy
            if model is not None:
                self.best_backup = self.backup_method(model)
            return True
        
        self.current += 1
        
        if self.current >= self.patience:
            return False
        return True

In [4]:
def train_one_epoch(model, optimizer, criterion, dataloader_train, training_device):
    for inputs, labels in dataloader_train:
        inputs, labels = inputs.to(training_device), labels.to(training_device)
        optimizer.zero_grad()
        loss = criterion(model(inputs), labels)
        loss.backward()
        optimizer.step()

def train_one_run(model, optimizer, criterion, dataloader_train, dataloader_val, max_epochs, early_stopper, trajectory, training_device='cuda', *_args, **_kwargs):
    model.train()
    model.to(training_device)

    for epoch in range(max_epochs):
        start_time = time.time()
        
        train_one_epoch(model, optimizer, criterion, dataloader_train, training_device)
        
        end_time = time.time()
        
        training_accuracy = eval_accuracy(model, dataloader_train, training_device)
        validation_accuracy = eval_accuracy(model, dataloader_val, training_device)
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "train": training_accuracy,
            "validation": validation_accuracy,
            "start_time": start_time,
            "duration": end_time - start_time,
            "memory_allocated_mb": torch.cuda.memory_allocated()/1024/1024,
            "memory_reserved_mb": torch.cuda.memory_reserved()/1024/1024,
        })
        
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            return early_stopper.best_backup
    
    return model

In [5]:
def run_aug_experiment(arch_name, model_factory, aug_name, aug_factory, run):
    path = "experiments/aug_" + aug_name + "_arch" + arch_name + "_" + run + "_"
    
    # 
    import os
    try:
        if os.stat(path + "report.json").st_size != 0:
            print("Report exists already for " + path[:-1] + ". Skipping...")
            return
    except OSError:
        pass
    
    train, test, val = aug_factory()
    model = model_factory()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    early_stopper = EarlyStopper(patience = 5)
    trajectory = []
    model = train_one_run(model, optimizer, criterion, train, val, 200, early_stopper, trajectory)
    validation_accuracy = eval_accuracy(model, val, "cuda")
    
    with open(path + "report.json", "w") as f:
        json.dump(
            {
                "name": arch_name,
                "run": run,
                "best_accuracy_validation": validation_accuracy,
                "time_generated": datetime.datetime.now().isoformat(),
                "trajectory": trajectory
            },
            f
        )
    torch.save(model, path + "model.pt")

In [6]:
def vgg_from_cfg(cfg_key):
    return VGG(
        make_vgg11_bn_layers(cfg=vgg_cfg[cfg_key]),
        num_classes=10,
        avgpool_size=(1,1)
    ).to("cuda")

def vgg11():
    return vgg_from_cfg("11")

def vgg13():
    return vgg_from_cfg("13")

def vgg16():
    return vgg_from_cfg("16")

In [7]:
def flip():
    return augmented_cifar10_dataset_randomflip(bs=128)

def small_rotate():
    return augmented_cifar10_dataset_randomrotate(5, bs=128)

def medium_rotate():
    return augmented_cifar10_dataset_randomrotate(45, bs=128)

def full_rotate():
    return augmented_cifar10_dataset_randomrotate(180, bs=128)

def fliprotate():
    return augmented_cifar10_dataset_randomflip_rotate(5, bs=128)

def noise():
    return augmented_cifar10_dataset_gauss_noise(0.05, bs=128)

def cutout():
    return augmented_cifar10_dataset_cutout(5, bs=128)

In [12]:
experiment_list = [
    (archfactory.__name__, archfactory, augfactory.__name__, augfactory, str(run))
    for run in range(1, 11)
    for archfactory in [vgg11, vgg13, vgg16]
    for augfactory in [flip, small_rotate,  fliprotate, noise]
] + [
    ("vgg11", vgg11, augfactory.__name__, augfactory, str(run))
    for run in range(1, 11)
    for augfactory in [medium_rotate, full_rotate]
] 

In [14]:
for experiment in experiment_list:
    print(
        "Time:", datetime.datetime.now().isoformat(), 
        "Arch: ", experiment[0], 
        "Aug: ", experiment[2],
        "Run: ", experiment[4]
    )
    run_aug_experiment(*experiment)

Time: 2023-03-19T18:48:10.683612 Arch:  vgg11 Aug:  flip Run:  1
Report exists already for experiments/aug_flip_archvgg11_1. Skipping...
Time: 2023-03-19T18:48:10.684887 Arch:  vgg11 Aug:  small_rotate Run:  1
Report exists already for experiments/aug_small_rotate_archvgg11_1. Skipping...
Time: 2023-03-19T18:48:10.686322 Arch:  vgg11 Aug:  fliprotate Run:  1
Report exists already for experiments/aug_fliprotate_archvgg11_1. Skipping...
Time: 2023-03-19T18:48:10.686621 Arch:  vgg11 Aug:  noise Run:  1
Report exists already for experiments/aug_noise_archvgg11_1. Skipping...
Time: 2023-03-19T18:48:10.686799 Arch:  vgg13 Aug:  flip Run:  1
Report exists already for experiments/aug_flip_archvgg13_1. Skipping...
Time: 2023-03-19T18:48:10.686949 Arch:  vgg13 Aug:  small_rotate Run:  1
Report exists already for experiments/aug_small_rotate_archvgg13_1. Skipping...
Time: 2023-03-19T18:48:10.687113 Arch:  vgg13 Aug:  fliprotate Run:  1
Report exists already for experiments/aug_fliprotate_archvgg1

Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, Accuracy on validation set: 0.3202
Epoch: 1, Accuracy on validation set: 0.5152
Epoch: 2, Accuracy on validation set: 0.6352
Epoch: 3, Accuracy on validation set: 0.7076
Epoch: 4, Accuracy on validation set: 0.7362
Epoch: 5, Accuracy on validation set: 0.7672
Epoch: 6, Accuracy on validation set: 0.7788
Epoch: 7, Accuracy on validation set: 0.7966
Epoch: 8, Accuracy on validation set: 0.8046
Epoch: 9, Accuracy on validation set: 0.8048
Epoch: 10, Accuracy on validation set: 0.8178
Epoch: 11, Accuracy on validation set: 0.8226
Epoch: 12, Accuracy on validation set: 0.828
Epoch: 13, Accuracy on validation set: 0.8356
Epoch: 14, Accuracy on validation set: 0.8294
Epoch: 15, Accuracy on validation set: 0.8402
Epoch: 16, Accuracy on validation set: 0.8366
Epoch: 17, Accuracy on validation set: 0.8434
Epoch: 18, Accuracy on validation set: 0.8456
Epoch: 19, Accuracy on validation set: 0.841
Epoch: 20, Accur

Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, Accuracy on validation set: 0.2606
Epoch: 1, Accuracy on validation set: 0.4302
Epoch: 2, Accuracy on validation set: 0.5628
Epoch: 3, Accuracy on validation set: 0.652
Epoch: 4, Accuracy on validation set: 0.695
Epoch: 5, Accuracy on validation set: 0.739
Epoch: 6, Accuracy on validation set: 0.7618
Epoch: 7, Accuracy on validation set: 0.79
Epoch: 8, Accuracy on validation set: 0.797
Epoch: 9, Accuracy on validation set: 0.8046
Epoch: 10, Accuracy on validation set: 0.8186
Epoch: 11, Accuracy on validation set: 0.8176
Epoch: 12, Accuracy on validation set: 0.8158
Epoch: 13, Accuracy on validation set: 0.8248
Epoch: 14, Accuracy on validation set: 0.831
Epoch: 15, Accuracy on validation set: 0.8298
Epoch: 16, Accuracy on validation set: 0.829
Epoch: 17, Accuracy on validation set: 0.8336
Epoch: 18, Accuracy on validation set: 0.8326
Epoch: 19, Accuracy on validation set: 0.8338
Epoch: 20, Accuracy on

Epoch: 6, Accuracy on validation set: 0.6398
Epoch: 7, Accuracy on validation set: 0.6734
Epoch: 8, Accuracy on validation set: 0.6928
Epoch: 9, Accuracy on validation set: 0.7166
Epoch: 10, Accuracy on validation set: 0.709
Epoch: 11, Accuracy on validation set: 0.7344
Epoch: 12, Accuracy on validation set: 0.7526
Epoch: 13, Accuracy on validation set: 0.7492
Epoch: 14, Accuracy on validation set: 0.7758
Epoch: 15, Accuracy on validation set: 0.7634
Epoch: 16, Accuracy on validation set: 0.7908
Epoch: 17, Accuracy on validation set: 0.7894
Epoch: 18, Accuracy on validation set: 0.7906
Epoch: 19, Accuracy on validation set: 0.7924
Epoch: 20, Accuracy on validation set: 0.8
Epoch: 21, Accuracy on validation set: 0.8122
Epoch: 22, Accuracy on validation set: 0.8174
Epoch: 23, Accuracy on validation set: 0.7988
Epoch: 24, Accuracy on validation set: 0.8172
Epoch: 25, Accuracy on validation set: 0.824
Epoch: 26, Accuracy on validation set: 0.8272
Epoch: 27, Accuracy on validation set: 0.82

Epoch: 22, Accuracy on validation set: 0.7608
Epoch: 23, Accuracy on validation set: 0.7642
Early stop
Time: 2023-03-19T22:12:26.737987 Arch:  vgg13 Aug:  flip Run:  7
Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, Accuracy on validation set: 0.3568
Epoch: 1, Accuracy on validation set: 0.5466
Epoch: 2, Accuracy on validation set: 0.6646
Epoch: 3, Accuracy on validation set: 0.706
Epoch: 4, Accuracy on validation set: 0.746
Epoch: 5, Accuracy on validation set: 0.7762
Epoch: 6, Accuracy on validation set: 0.7978
Epoch: 7, Accuracy on validation set: 0.8072
Epoch: 8, Accuracy on validation set: 0.823
Epoch: 9, Accuracy on validation set: 0.8322
Epoch: 10, Accuracy on validation set: 0.8416
Epoch: 11, Accuracy on validation set: 0.8412
Epoch: 12, Accuracy on validation set: 0.8448
Epoch: 13, Accuracy on validation set: 0.8462
Epoch: 14, Accuracy on validation set: 0.8436
Epoch: 15, Accuracy on validation set: 0.852
Epoch: 16, Accuracy on validation 

...

Time: 2023-03-20T07:25:43.058607 Arch:  vgg13 Aug:  noise Run:  10
Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, Accuracy on validation set: 0.2576
Epoch: 1, Accuracy on validation set: 0.4458
Epoch: 2, Accuracy on validation set: 0.5592
Epoch: 3, Accuracy on validation set: 0.6092
Epoch: 4, Accuracy on validation set: 0.6622
Epoch: 5, Accuracy on validation set: 0.6964
Epoch: 6, Accuracy on validation set: 0.7218
Epoch: 7, Accuracy on validation set: 0.7372
Epoch: 8, Accuracy on validation set: 0.7518
Epoch: 9, Accuracy on validation set: 0.757
Epoch: 10, Accuracy on validation set: 0.7754
Epoch: 11, Accuracy on validation set: 0.7756
Epoch: 12, Accuracy on validation set: 0.7846
Epoch: 13, Accuracy on validation set: 0.7864
Epoch: 14, Accuracy on validation set: 0.7808
Epoch: 15, Accuracy on validation set: 0.7844
Epoch: 16, Accuracy on validation set: 0.7816
Epoch: 17, Accuracy on validation set: 0.7852
Epoch: 18, Accuracy on validation set: 0

Epoch: 11, Accuracy on validation set: 0.748
Epoch: 12, Accuracy on validation set: 0.7318
Epoch: 13, Accuracy on validation set: 0.763
Epoch: 14, Accuracy on validation set: 0.7578
Epoch: 15, Accuracy on validation set: 0.7728
Epoch: 16, Accuracy on validation set: 0.774
Epoch: 17, Accuracy on validation set: 0.767
Epoch: 18, Accuracy on validation set: 0.7768
Epoch: 19, Accuracy on validation set: 0.78
Epoch: 20, Accuracy on validation set: 0.7862
Epoch: 21, Accuracy on validation set: 0.794
Epoch: 22, Accuracy on validation set: 0.7866
Epoch: 23, Accuracy on validation set: 0.7784
Epoch: 24, Accuracy on validation set: 0.781
Epoch: 25, Accuracy on validation set: 0.7812
Epoch: 26, Accuracy on validation set: 0.7864
Early stop
Time: 2023-03-20T08:49:33.338371 Arch:  vgg11 Aug:  medium_rotate Run:  1
Report exists already for experiments/aug_medium_rotate_archvgg11_1. Skipping...
Time: 2023-03-20T08:49:33.338757 Arch:  vgg11 Aug:  full_rotate Run:  1
Report exists already for experime

Epoch: 10, Accuracy on validation set: 0.5826
Epoch: 11, Accuracy on validation set: 0.6008
Epoch: 12, Accuracy on validation set: 0.5934
Epoch: 13, Accuracy on validation set: 0.6158
Epoch: 14, Accuracy on validation set: 0.6348
Epoch: 15, Accuracy on validation set: 0.6378
Epoch: 16, Accuracy on validation set: 0.6328
Epoch: 17, Accuracy on validation set: 0.6496
Epoch: 18, Accuracy on validation set: 0.6466
Epoch: 19, Accuracy on validation set: 0.6596
Epoch: 20, Accuracy on validation set: 0.6668
Epoch: 21, Accuracy on validation set: 0.6714
Epoch: 22, Accuracy on validation set: 0.6752
Epoch: 23, Accuracy on validation set: 0.6808
Epoch: 24, Accuracy on validation set: 0.6846
Epoch: 25, Accuracy on validation set: 0.6882
Epoch: 26, Accuracy on validation set: 0.6964
Epoch: 27, Accuracy on validation set: 0.7
Epoch: 28, Accuracy on validation set: 0.7008
Epoch: 29, Accuracy on validation set: 0.7002
Epoch: 30, Accuracy on validation set: 0.699
Epoch: 31, Accuracy on validation set:

Epoch: 10, Accuracy on validation set: 0.593
Epoch: 11, Accuracy on validation set: 0.5922
Epoch: 12, Accuracy on validation set: 0.6136
Epoch: 13, Accuracy on validation set: 0.6166
Epoch: 14, Accuracy on validation set: 0.6316
Epoch: 15, Accuracy on validation set: 0.6384
Epoch: 16, Accuracy on validation set: 0.6508
Epoch: 17, Accuracy on validation set: 0.6546
Epoch: 18, Accuracy on validation set: 0.6586
Epoch: 19, Accuracy on validation set: 0.6818
Epoch: 20, Accuracy on validation set: 0.6816
Epoch: 21, Accuracy on validation set: 0.676
Epoch: 22, Accuracy on validation set: 0.6728
Epoch: 23, Accuracy on validation set: 0.6744
Epoch: 24, Accuracy on validation set: 0.6808
Early stop
Time: 2023-03-20T10:53:49.440548 Arch:  vgg11 Aug:  medium_rotate Run:  8
Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, Accuracy on validation set: 0.2642
Epoch: 1, Accuracy on validation set: 0.396
Epoch: 2, Accuracy on validation set: 0.481
Epoch: 3, Accuracy

Epoch: 37, Accuracy on validation set: 0.724
Epoch: 38, Accuracy on validation set: 0.7116
Epoch: 39, Accuracy on validation set: 0.7194
Epoch: 40, Accuracy on validation set: 0.7338
Epoch: 41, Accuracy on validation set: 0.7264
Epoch: 42, Accuracy on validation set: 0.7276
Epoch: 43, Accuracy on validation set: 0.7322
Epoch: 44, Accuracy on validation set: 0.7274
Epoch: 45, Accuracy on validation set: 0.7324
Early stop
Time: 2023-03-20T11:56:47.781868 Arch:  vgg11 Aug:  medium_rotate Run:  10
Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, Accuracy on validation set: 0.3278
Epoch: 1, Accuracy on validation set: 0.4438
Epoch: 2, Accuracy on validation set: 0.5106
Epoch: 3, Accuracy on validation set: 0.607
Epoch: 4, Accuracy on validation set: 0.6282
Epoch: 5, Accuracy on validation set: 0.6614
Epoch: 6, Accuracy on validation set: 0.6768
Epoch: 7, Accuracy on validation set: 0.7072
Epoch: 8, Accuracy on validation set: 0.7134
Epoch: 9, Accuracy on

In [15]:
len(experiment_list)

140